#**Maestría en Inteligencia Artificial Aplicada**
##**Proyecto Integrador**
###Tecnológico de Monterrey
###Prof Gretel Barceló Alonso

### **Semana 4**
####**Avance 2: Ingenieria de Caracteristicas**
# Equipo 1
#**Rodrigo Ruiz Gonzalez Matricula : A01793081**
#**Daniel Hernandez Mora Matricula : A01793538**
#**Juan Sebastián Téllez López Matricula : A01793859**

# **Ingeniería de caracteristicas**

En el siguiente notebook vamos a realizar la preparación de nuestro conjunto de entrenamiento, teniendo en cuenta técnicas que nos permitan ajustar las caracteristicas, así como el conocimiento de negocio

#**Importamos librerias**

In [ ]:
import pandas as pd
import datetime
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

#**Importamos nuestro Dataset de Entrada**

In [ ]:
#Archivo unificado y validado por cliente#
df = pd.read_csv("input_final.csv", sep=',', encoding ="latin-1")
df.head(5)

,DOCUMENTO,PRIORIDAD,MOTIVO_NO_PAGO,DESC_ACTIVIDAD,GENERO,EDAD,HORA,HORA_OK,RANGO_HORA,RANGO_HORA2,...,PRIM_CTA,MORA_TEMPRANA,MANTENIDO,ACCION_MEJOR_CONTACTO,EFECTO_MEJOR_CONTACTO,NEGOCIACION_ACTIVA,CANTIDAD_GESTIONES,GESTION_LLAMADAS,CONTEO_CONTACTOS_DIRECTOS,MARCA_TIPO_CARTERA
0,11000000821,5,NO MOTIVO,Â Asalariados.,F,23,15:45:12,15,Tarde,Tarde,...,X,X,X,HACER LLAMADA,YA PAGO,NO,9.0,1.0,1.0,PROD_DIG
1,11000000821,5,NO MOTIVO,Â Asalariados.,F,23,9:50:27,9,Mañana,Mañana,...,X,X,X,HACER LLAMADA,YA PAGO,NO,9.0,1.0,1.0,PROD_DIG
2,11000000821,21,ASA_REDUCCION_INGRESOS,Â Asalariados.,F,23,9:49:51,9,Mañana,Mañana,...,X,X,X,HACER LLAMADA,YA PAGO,NO,9.0,1.0,1.0,PROD_DIG
3,11000000821,5,SIN_ACCESO_A_CANALES,Â Asalariados.,F,23,10:02:47,10,Mañana,Mañana,...,X,X,X,HACER LLAMADA,YA PAGO,NO,9.0,1.0,1.0,PROD_DIG
4,11000000821,5,ASA_REDUCCION_INGRESOS,Â Asalariados.,F,23,14:23:01,14,Tarde,Mediodia,...,X,X,X,HACER LLAMADA,YA PAGO,NO,9.0,1.0,1.0,PROD_DIG


#**Separación de nuestro dataset**

A continuación vamos a dividir nuestro input en 3 conjuntos diferentes:

- El primero seran datos identificadores (Nombres, documentos, numeros de telefono etc.) Los cuales no seran parte del conjunto de entrenamiento sino que seran utilizados para agregar la etiqueta de las predicciones.
- El segundo conjunto seran todas las variables que utilizaremos para el preprocesamiento de datos y en las cuales aplicaremos las técnicas correspondientes.
- El tercero se refiere a nuestras variables predictoras, ya que vamos a tener diferentes formas de mostrala para la realización de diversos experimentos desde varios enfoques.  


##**Identificadores**

In [ ]:
# Mostramos la lista de los nombres de todas las columnas de nuestro dataset#
df.columns

Index(['DOCUMENTO', 'PRIORIDAD', 'MOTIVO_NO_PAGO', 'DESC_ACTIVIDAD', 'GENERO',
       'EDAD', 'HORA', 'HORA_OK', 'RANGO_HORA', 'RANGO_HORA2',
       'SEGMENTO_CLIENTE', 'RIESGO_ACTUAL', 'RECUPERACION', 'MORA_INICIAL',
       'MORA_ACTUAL', 'C_RIESGO', 'ESTADO_ICS_ACTUAL', 'DIAS_ACTUAL',
       'CICLO_OBLIG', 'CIUDAD', 'CED_SIN_DIG', 'NOMBRES', 'SUB_PRODUCTO',
       'SALDO_OBLIG', 'MORA_OBLIG', 'DIAS_OBLIG', 'TIPO_CLIENTE', 'SALDO_CL',
       'RANGO_SALDO', 'DIAS_CL', 'RANGO_CL', 'SCORE', 'F_APERTURA', 'PRIM_CTA',
       'MORA_TEMPRANA', 'MANTENIDO', 'ACCION_MEJOR_CONTACTO',
       'EFECTO_MEJOR_CONTACTO', 'NEGOCIACION_ACTIVA', 'CANTIDAD_GESTIONES',
       'GESTION_LLAMADAS', 'CONTEO_CONTACTOS_DIRECTOS', 'MARCA_TIPO_CARTERA'],
      dtype='object')

seleccionamos tres columnas específicas ('DOCUMENTO', 'CED_SIN_DIG' y 'NOMBRES') de los nombres de las columanas y las asignamos a un nuevo DataFrame llamado identificadores

In [ ]:
# Seleccionamos las columnas 'DOCUMENTO', 'CED_SIN_DIG' y 'NOMBRES' del DataFrame df y las asigna a un nuevo DataFrame llamado identificadores#
# Utilizando el método loc[] de pandas para seleccionar filas y columnas#
# basadas en etiquetas#
identificadores = df.loc[:,['DOCUMENTO','CED_SIN_DIG','NOMBRES']]
identificadores.head(5)

,DOCUMENTO,CED_SIN_DIG,NOMBRES
0,11000000821,1000000821,ANA . MALAVER RODRIGUEZ
1,11000000821,1000000821,ANA . MALAVER RODRIGUEZ
2,11000000821,1000000821,ANA . MALAVER RODRIGUEZ
3,11000000821,1000000821,ANA . MALAVER RODRIGUEZ
4,11000000821,1000000821,ANA . MALAVER RODRIGUEZ


##**Caracteristicas**

Las características seleccionadas son : 'PRIORIDAD','MOTIVO_NO_PAGO','DESC_ACTIVIDAD','GENERO','EDAD','SEGMENTO_CLIENTE','RIESGO_ACTUAL',#'RECUPERACION,'MORA_INICIAL','MORA_ACTUAL','C_RIESGO','ESTADO_ICS_ACTUAL',#'DIAS_ACTUAL','CICLO_OBLIG','CIUDAD','SUB_PRODUCTO','SALDO_OBLIG','MORA_OBLIG',#'DIAS_OBLIG','TIPO_CLIENTE','SALDO_CL','RANGO_SALDO','DIAS_CL','RANGO_CL','SCORE','F_APERTURA','PRIM_CTA','MORA_TEMPRANA','MANTENIDO','ACCION_MEJOR_CONTACTO','EFECTO_MEJOR_CONTACTO','NEGOCIACION_ACTIVA','CANTIDAD_GESTIONES','GESTION_LLAMADAS','CONTEO_CONTACTOS_DIRECTOS'y 'MARCA_TIPO_CARTERA'#

In [ ]:
# seleccionamos un conjunto específico de características o variables de#
# interés# del nuestro dataset de entrada y lo asignamos a un nuevo DataFrame#
# llamado features#
features = df.loc[:,['PRIORIDAD', 'MOTIVO_NO_PAGO', 'DESC_ACTIVIDAD', 'GENERO','EDAD','SEGMENTO_CLIENTE', 'RIESGO_ACTUAL', 'RECUPERACION', 'MORA_INICIAL',
                     'MORA_ACTUAL', 'C_RIESGO', 'ESTADO_ICS_ACTUAL', 'DIAS_ACTUAL','CICLO_OBLIG', 'CIUDAD', 'SUB_PRODUCTO','SALDO_OBLIG', 'MORA_OBLIG',
                     'DIAS_OBLIG', 'TIPO_CLIENTE', 'SALDO_CL','RANGO_SALDO', 'DIAS_CL', 'RANGO_CL', 'SCORE', 'F_APERTURA', 'PRIM_CTA','MORA_TEMPRANA',
                     'MANTENIDO', 'ACCION_MEJOR_CONTACTO','EFECTO_MEJOR_CONTACTO', 'NEGOCIACION_ACTIVA', 'CANTIDAD_GESTIONES','GESTION_LLAMADAS',
                     'CONTEO_CONTACTOS_DIRECTOS', 'MARCA_TIPO_CARTERA']]
features.head(5)

,PRIORIDAD,MOTIVO_NO_PAGO,DESC_ACTIVIDAD,GENERO,EDAD,SEGMENTO_CLIENTE,RIESGO_ACTUAL,RECUPERACION,MORA_INICIAL,MORA_ACTUAL,...,PRIM_CTA,MORA_TEMPRANA,MANTENIDO,ACCION_MEJOR_CONTACTO,EFECTO_MEJOR_CONTACTO,NEGOCIACION_ACTIVA,CANTIDAD_GESTIONES,GESTION_LLAMADAS,CONTEO_CONTACTOS_DIRECTOS,MARCA_TIPO_CARTERA
0,5,NO MOTIVO,Â Asalariados.,F,23,MONO_TC,NaN,NORMALIZADO,30,0,...,X,X,X,HACER LLAMADA,YA PAGO,NO,9.0,1.0,1.0,PROD_DIG
1,5,NO MOTIVO,Â Asalariados.,F,23,MONO_TC,NaN,NORMALIZADO,30,0,...,X,X,X,HACER LLAMADA,YA PAGO,NO,9.0,1.0,1.0,PROD_DIG
2,21,ASA_REDUCCION_INGRESOS,Â Asalariados.,F,23,MONO_TC,NaN,NORMALIZADO,30,0,...,X,X,X,HACER LLAMADA,YA PAGO,NO,9.0,1.0,1.0,PROD_DIG
3,5,SIN_ACCESO_A_CANALES,Â Asalariados.,F,23,MONO_TC,NaN,NORMALIZADO,30,0,...,X,X,X,HACER LLAMADA,YA PAGO,NO,9.0,1.0,1.0,PROD_DIG
4,5,ASA_REDUCCION_INGRESOS,Â Asalariados.,F,23,MONO_TC,NaN,NORMALIZADO,30,0,...,X,X,X,HACER LLAMADA,YA PAGO,NO,9.0,1.0,1.0,PROD_DIG


##**Variables predictoras**

Las variables predictoras, también conocidas como variables independientes o características, son aquellas que se utilizan en un modelo de aprendizaje automático para predecir o explicar el valor de una variable objetivo.

In [ ]:
# Seleccionamos nuestras variables predictoras compuestas de cuatro columnas#
# específicas# ('HORA', 'HORA_OK', 'RANGO_HORA', 'RANGO_HORA2') de nuestro#
# Dataset de# entrada y se asigna a un nuevo DataFrame llamado predictoras#
predictoras = df.loc[:,['HORA', 'HORA_OK', 'RANGO_HORA', 'RANGO_HORA2']]
predictoras.head(5)


,HORA,HORA_OK,RANGO_HORA,RANGO_HORA2
0,15:45:12,15,Tarde,Tarde
1,9:50:27,9,Mañana,Mañana
2,9:49:51,9,Mañana,Mañana
3,10:02:47,10,Mañana,Mañana
4,14:23:01,14,Tarde,Mediodia


#**Ajustar tipo de datos y posibles valores vacios en nuestras features**

Una vez separadas nuestras variables de interes para el modelo, vamos a iniciar por una validación de posibles valores vacios y ajuste del tipo de datos

In [ ]:
# Llenar RIESGO_ACTUAL con 0 y NO_DEF
features["RIESGO_ACTUAL"].fillna("0 NO_DEF", inplace=True)

# Redondear C_RIESGO a número entero
features["C_RIESGO"] = features["C_RIESGO"].astype(int)

# Llenar ESTADO_ICS_ACTUAL con NONE
features["ESTADO_ICS_ACTUAL"].fillna("NONE", inplace=True)

# Llenar DIAS_ACTUAL con 0
features["DIAS_ACTUAL"].fillna(0, inplace=True)

# Redondear DIAS_ACTUAL a número entero
features["DIAS_ACTUAL"] = features["DIAS_ACTUAL"].astype(int)

###################################################################################################

# Redondear SALDO_OBLIG a número entero
# Convertir valores a tipo float y manejar valores no numéricos como NaN
features["SALDO_OBLIG"] = pd.to_numeric(features["SALDO_OBLIG"], errors='coerce')

# Reemplazar valores NaN con 0
features["SALDO_OBLIG"].fillna(0, inplace=True)

# Redondear y convertir a tipo entero
features["SALDO_OBLIG"] = features["SALDO_OBLIG"].apply(lambda x: int(round(x)))

###################################################################################################

# Redondear MORA_OBLIG a número entero
# Convertir valores a tipo float y manejar valores no numéricos como NaN
features["MORA_OBLIG"] = pd.to_numeric(features["MORA_OBLIG"], errors='coerce')

# Reemplazar valores NaN con 0
features["MORA_OBLIG"].fillna(0, inplace=True)

# Redondear y convertir a tipo entero
features["MORA_OBLIG"] = features["MORA_OBLIG"].apply(lambda x: int(round(x)))

###################################################################################################

# Redondear DIAS_OBLIG a número entero
features["DIAS_OBLIG"] = features["DIAS_OBLIG"].astype(int)

###################################################################################################

# Convertir valores a tipo float y manejar valores no numéricos como NaN
features["SALDO_CL"] = pd.to_numeric(features["SALDO_CL"], errors='coerce')

# Reemplazar valores NaN con 0
features["SALDO_CL"].fillna(0, inplace=True)

# Redondear y convertir a tipo entero
features["SALDO_CL"] = features["MORA_OBLIG"].apply(lambda x: int(round(x)))

###################################################################################################

# Obtener el primer dígito de RANGO_CL
features["RANGO_CL"] = features["RANGO_CL"].apply(lambda row: int(str(row).split("_")[0]))

# Reemplazar "." por 0 en SCORE
features["SCORE"] = features["SCORE"].str.replace(".", "0")

# Cambiar el formato de F_APERTURA a DD/MM/YYYY
features["F_APERTURA"] = pd.to_datetime(features["F_APERTURA"], format='%d/%m/%Y', errors='coerce').dt.date

# Calcular la diferencia de días entre la fecha de hoy y F_APERTURA
fecha_actual = pd.Timestamp.now().normalize()  # Obtener la fecha actual en formato Timestamp
features["F_APERTURA"] = (fecha_actual - pd.to_datetime(features["F_APERTURA"])).dt.days

# Llenar valores nulos con 0
features["F_APERTURA"].fillna(0, inplace=True)

# Convertir a tipo entero
features["F_APERTURA"] = features["F_APERTURA"].astype(int)

# Redondear CANTIDAD_GESTIONES a número entero
features['CANTIDAD_GESTIONES'].fillna(0, inplace=True)
features["CANTIDAD_GESTIONES"] = features["CANTIDAD_GESTIONES"].apply(lambda x: int(round(x)))

# Redondear GESTION_LLAMADAS a número entero
features['GESTION_LLAMADAS'].fillna(0, inplace=True)
features["GESTION_LLAMADAS"] = features["GESTION_LLAMADAS"].apply(lambda x: int(round(x)))

# Redondear CONTEO_CONTACTOS_DIRECTOS a número entero
features['CONTEO_CONTACTOS_DIRECTOS'].fillna(0, inplace=True)
features["CONTEO_CONTACTOS_DIRECTOS"] = features["CONTEO_CONTACTOS_DIRECTOS"].apply(lambda x: int(round(x)))

features.head(5)

,PRIORIDAD,MOTIVO_NO_PAGO,DESC_ACTIVIDAD,GENERO,EDAD,SEGMENTO_CLIENTE,RIESGO_ACTUAL,RECUPERACION,MORA_INICIAL,MORA_ACTUAL,...,PRIM_CTA,MORA_TEMPRANA,MANTENIDO,ACCION_MEJOR_CONTACTO,EFECTO_MEJOR_CONTACTO,NEGOCIACION_ACTIVA,CANTIDAD_GESTIONES,GESTION_LLAMADAS,CONTEO_CONTACTOS_DIRECTOS,MARCA_TIPO_CARTERA
0,5,NO MOTIVO,Â Asalariados.,F,23,MONO_TC,0 NO_DEF,NORMALIZADO,30,0,...,X,X,X,HACER LLAMADA,YA PAGO,NO,9,1,1,PROD_DIG
1,5,NO MOTIVO,Â Asalariados.,F,23,MONO_TC,0 NO_DEF,NORMALIZADO,30,0,...,X,X,X,HACER LLAMADA,YA PAGO,NO,9,1,1,PROD_DIG
2,21,ASA_REDUCCION_INGRESOS,Â Asalariados.,F,23,MONO_TC,0 NO_DEF,NORMALIZADO,30,0,...,X,X,X,HACER LLAMADA,YA PAGO,NO,9,1,1,PROD_DIG
3,5,SIN_ACCESO_A_CANALES,Â Asalariados.,F,23,MONO_TC,0 NO_DEF,NORMALIZADO,30,0,...,X,X,X,HACER LLAMADA,YA PAGO,NO,9,1,1,PROD_DIG
4,5,ASA_REDUCCION_INGRESOS,Â Asalariados.,F,23,MONO_TC,0 NO_DEF,NORMALIZADO,30,0,...,X,X,X,HACER LLAMADA,YA PAGO,NO,9,1,1,PROD_DIG


In [ ]:
# tomamos el DataFrame features y lo guardamos en un archivo CSV llamado#
# "features.csv", sin incluir el índice del DataFrame, y utilizando la#
# codificación "latin-1#
features.to_csv("features.csv", index=False,encoding="latin-1")

##**Validación de que ya no tengamos valores faltantes**

Los valores faltantes pueden afectar la precisión de los análisis y modelos estadísticos o de aprendizaje automático. Esto se debe a que los algoritmos pueden no funcionar correctamente o pueden producir resultados sesgados si se encuentran con valores faltantes en los datos.

In [ ]:
# Calcular los valores faltantes
missing_data = features.isnull().sum()
missing_data_percentage = (missing_data / len(features)) * 100
missing_data_summary = pd.concat([missing_data, missing_data_percentage], axis=1, keys=['Total', 'Porcentaje'])
missing_data_df = pd.DataFrame(missing_data_summary, columns=['Total', 'Porcentaje'])

# Agregar un mensaje antes del DataFrame
total_records = len(features)
message = f"De un total de {total_records} registros se encuentran los siguientes valores faltantes:\n"
print(message)
missing_data_df.head(50)

De un total de 557729 registros se encuentran los siguientes valores faltantes:



,Total,Porcentaje
PRIORIDAD,0,0.0
MOTIVO_NO_PAGO,0,0.0
DESC_ACTIVIDAD,0,0.0
GENERO,0,0.0
EDAD,0,0.0
SEGMENTO_CLIENTE,0,0.0
RIESGO_ACTUAL,0,0.0
RECUPERACION,0,0.0
MORA_INICIAL,0,0.0
MORA_ACTUAL,0,0.0


#**Generación de nuevas caracteristicas**

A continuación se dará una justificación detallada de que transformaciones se realizarán, cuales no seran necesesarias y si habra exclusión de variables por algún motivo:

- PRIORIDAD: Solo basta con llenar los vacios en 0, ya que si no tienen un valor asignado no requieren una gestión inmediata.
- MOTIVO_NO_PAGO: Vacios se reemplazaron por NO MOTIVO, desde el archivo de unificación de bases, no se realizarán agrupaciones ya que no son tantas categorias que puedan sobredimensionar esta categoria.
- DESC_ACTIVIDAD: Se requiere hacer una agrupación por categorias un poco mas generales, por ejemplo en un grupo todas las relacionadas con comercio, otras con transporte, otras con educación, ya que hay categorias en las que solo aplica un porcentaje menor al 1% de clientes y esto puede sobredimensionar el entrenamiento del modelo innecesariamente.
- GENERO: Previamente en el notebook de unificación de bases se garantizo que los valores vacios fueran reemplazados por D, lo cual indica que no son definidos, no necesitamos realizar ninguna transformación adicional.
- EDAD: Previamente en el notebook de unificación de bases a partir de la fecha de nacimiento en formato dd/mm/aaaa se hizo la diferencia al día actual para obtener la edad del cliente, por lo tanto no necesitamos ninguna transformación adicional.
- SEGMENTO: Se deja tal cual, ya que solo vienen dos categorias MONO_TC que indica que el cliente solo tiene un producto financiero y MULTI_TC que indica que el cliente tiene mas de un producto financiero.
- RIESGO ACTUAL: Actualmente viene con el estilo 0 NO DEF 4 MEDIO 6 ALTO etc... De esta columna vamos a tomar los numeros nada más, ya que entre mayor es más el riesgo y no impacta en nada el quitar la etiqueta de la clasificación.
- RECUPERACIÓN: Se dejaran las categorias como vienen, y más adelante se les aplicará la respectiva codificación para variables categoricas.
- MORA INICIAL y MORA ACTUAL, DIAS_ACTUAL, DIAS_OBLIG: Se dejara tal cual, ya que es la clasificación que tiene al cliente respecto a los días en mora.
- C_RIESGO, SCORE: Se eliminaran valores negativos que vienen de clasificaciones incorrectas por parte del cliente y se dejaran en 0, al no conocer de que manera es calculada.
- ESTADO_ICS_ACTUAL: Este es un dato proveniente del estado actual de gestión con el cliente, el cual viene por defecto en ingles, pero no afecta en su posterior codificación como variable categorica.
- CICLO_OBLIG: Se deja tal cual, posteriormente se evaluara si se excluye dependiendo su impacto en el entrenamiento del modelo, este es un dato valioso para la operación ya que permite identificar que día el cliente debe realizar el pago.
- CIUDAD: Es una caracteristica de bastante importancia para el modelo, pero se debe evaluar cuidadosamente la manera en que va a ser codificada ya que no es posible mediante un one hot encoder debido al alto numero de categorias, por lo tanto se optara inicialmente por un Target encoding o una codificación de frecuencia agrupada, para evitar generar columnas adicionales que sobrecarguen el entrenamiento.
- SUB_PRODUCTO: No tiene tantas categorias y de momento no se evalua la posibilidad de agrupar por franquicia ya que este campo nos especifica el tipo de producto que tiene el cliente EJ Visa Platinum, y esto puede permitir una mejor clasificación de los clientes.
- SALDO_OBLIG, MORA_OBLIG: Corresponden a saldos que se dejaran en tipo entero, si bien no se realiza un ordenamiento por los saldos, si es una medición importante de la cartera a recuperar.
- TIPO_CLIENTE: Es una clasificación muy importante que nos permite conocer un poco más sobre el comportamiento financiero del cliente.
- SALDO_CL: Sera excluida ya que contiene la misma información de SALDO_OBLIG.
- RANGO_SALDO: Contiene los saldos dentro de unos rangos, los cuales son un poco amplios, sera excluida ya que se cuenta con el campo RANGO_CL el cual enumera dichos rangos de saldo y ya ahorra realizar una transformación adicional.
- DIAS_CL: Sera excluida, ya que es un duplicado de DIAS_OBLIG.
- F_APERTURA: Previamente se hizo una transformación para tener la cantidad de días transcurridos desde la creación de la cuenta, lo cual nos daría la antiguedad del cliente y podría ayudar a la clasificación del mismo.
- PRIM_CTA, MORA_TEMPRANA, MANTENIDO: Se excluiran del modelo, ya que hacen referencia a estados de cartera en los cuales se encuentra el cliente, y para esta base todos hacen parte de un mismo segmento de clientes (Cartera reciente) que son clientes que no tienen estados de mora muy amplios.
- ACCION_MEJOR_CONTACTO, EFECTO_MEJOR_CONTACTO: Se les aplicara la respectiva codificación one hot encoding, ya que no tienen tantas categorias y es información historica relevante sobre gestiones previas que ha tenido el cliente.
- NEGOCIACION_ACTIVA: Nos indica si el cliente se encuentra en algun proceso para la gestión del cobro, por lo tanto se le aplicara la respectiva codificación one hot encoding.
- CANTIDAD_GESTIONES, GESTION_LLAMADAS, CONTEO_CONTACTOS_DIRECTOS: Referentes a intentos previos realizados al cliente, se les aplicará la respectiva codificación.
- MARCA_TIPO_CARTERA: Referente a si el cliente tiene algun beneficio por parte del estado, lo cual alivianaria el momento de la negociación para la recuperación de la cartera

In [ ]:
# Codificación de target
target_encoded = df.groupby('ciudad')['target'].mean()
df['ciudad_encoded'] = df['ciudad'].map(target_encoded)

# Codificación de frecuencia agrupada
top_cities = df['ciudad'].value_counts().head(10).index  # Supongamos que tomamos las 10 ciudades más frecuentes
df['ciudad_encoded'] = df['ciudad'].apply(lambda x: x if x in top_cities else 'Otros')

# Codificación de frecuencia
frequency_encoded = df['ciudad'].value_counts(normalize=True)
df['ciudad_encoded'] = df['ciudad'].map(frequency_encoded)